In [ ]:
# setting device type:
# https://www.tensorflow.org/api_docs/python/tf/config/set_visible_devices
# https://www.tensorflow.org/guide/gpu
# https://stackoverflow.com/a/59483305/312594
# https://stackoverflow.com/questions/37660312/how-to-run-tensorflow-on-cpu
# https://stackoverflow.com/questions/59411681/tensorflow-2-how-to-switch-execution-from-gpu-to-cpu-and-back

from types import ModuleType as MT

def reportImports():
    # https://stackoverflow.com/questions/4858100/how-to-list-imported-modules
    all = [k for k,v in globals().items() if type(v) is MT and not k.startswith('__')]
    imports = ", ".join(all)
    print(f'imports: [{imports}]')

import tensorflow as tf
import timeit

print(f'tensorflow version: {tf.__version__}')

# To find out which devices your operations and tensors are assigned to
#tf.debugging.set_log_device_placement(True)

useCPU = False
hideGPUs = useCPU and False

cpuDevice = None
gpuDevice = None

for device in tf.config.list_physical_devices():
    name = '/' + device.name.lstrip('/physical_device:');

    print(f'{device.device_type} device found at {name}')

    if device.device_type == 'CPU':
        cpuDevice = name
    if device.device_type == 'GPU':
        gpuDevice = name

if hideGPUs:
    print('hiding GPUs')
    tf.config.set_visible_devices([], 'GPU')

if useCPU or gpuDevice is None:
    selectedDevice = cpuDevice
else:
    selectedDevice = gpuDevice

print(f'running tensorflow on {selectedDevice}')

with tf.device(selectedDevice):
    print('setting loss_fn')
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

    print('setting cifar')
    cifar = tf.keras.datasets.cifar100

    print('calling cifar.load_data()')
    (x_train, y_train), (x_test, y_test) = cifar.load_data()

    print('setting model')
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        input_shape=(32, 32, 3),
        classes=100,)

    print('compiling model')
    %timeit model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

    print('fitting model')
    %timeit model.fit(x_train, y_train, epochs=5, batch_size=64)

print('done')


tensorflow version: 2.15.0
CPU device found at /CPU:0
GPU device found at /GPU:0
running tensorflow on /GPU:0
setting loss_fn
setting cifar
calling cifar.load_data()
setting model


2024-02-20 19:58:36.611260: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-02-20 19:58:36.611280: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-02-20 19:58:36.611284: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-02-20 19:58:36.611327: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-20 19:58:36.611340: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


compiling model
3.45 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
fitting model
Epoch 1/5


2024-02-20 19:58:42.410073: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


209/782 [=======>......................] - ETA: 38s - loss: 5.1641 - accuracy: 0.0316